In [1]:
import modules.models.eca_mobilenetv2 as model

import numpy as np
import torch
from torchinfo import summary

import matplotlib.pyplot as plt

# Profiler: torchprofile

In [2]:
from torchprofile import profile_macs

# Profiler: torchlop

In [3]:
from torchlop import profile

In [4]:
def get_macs_and_params_per_layer(model):

    dummy_inp = torch.randn(1, 3, 224, 224)
    macs, params, layer_infos = profile(model, inputs=(dummy_inp, ))

    print('Total MACs of {}: {:.4g} M'.format(model.__class__.__name__, macs / 1e6))
    print('Total params of {}: {}'.format(model.__class__.__name__, params))

    macs_layer = {}
    params_layer = {}

    conv_idx = 0
    linear_idx = 0
    
    for i, k in enumerate(layer_infos.keys()):
        layer_type = layer_infos[k]["type"]
        layer_ops = layer_infos[k]["ops"]
        layer_params = layer_infos[k]["params"]
        if layer_ops != 0 and layer_type != "BatchNorm2d":
            if layer_type == "Conv2d": 
                macs_layer.update({layer_type + '_' + str(conv_idx): layer_ops})   
                params_layer.update({layer_type + '_' + str(conv_idx): layer_params}) 
                conv_idx += 1
            elif layer_type == "Linear":
                macs_layer.update({layer_type + '_' + str(linear_idx): layer_ops}) 
                params_layer.update({layer_type + '_' + str(linear_idx): layer_params})  
                linear_idx += 1 
            else:
                macs_layer.update({layer_type: layer_ops})
                params_layer.update({layer_type: layer_params})

    return macs_layer, params_layer

## Plot results

In [5]:
def plot_results(res_layer, res_type: str):
    fig, ax = plt.subplots(figsize=(10,3))

    layers_name = list(res_layer.keys())
    layer_res = list(res_layer.values())
    
    rects = ax.bar(layers_name, layer_res)
    ax.bar_label(rects, padding=3, fontsize=7)
    if res_type == "macs":
        ylabel = "MACs (M)"
        title = "MACs per layer (M)"
    elif res_type == "params":
        ylabel = "#Params"  
        title = "#Params per layer"
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(np.arange(len(layers_name)), layers_name, rotation=90, fontsize=8)
    
    plt.show()

# Models

In [6]:
eca_model = model.ECA_MobileNetV2().to('cpu')

In [7]:
eca_model.eval();

## MobileNetV2

### Profile

In [8]:
inputs = torch.randn(1, 3, 224, 224)

macs = profile_macs(eca_model, inputs)
print('{}: {:.4g} M'.format(eca_model.__class__.__name__, macs / 1e6))

ECA_MobileNetV2: 44.13 M


/opt/conda/envs/pytorch_23/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::unsqueeze". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/opt/conda/envs/pytorch_23/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::expand_as". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


In [9]:
macs_mbnet, params_mbnet = get_macs_and_params_per_layer(eca_model)

Total MACs of ECA_MobileNetV2: 46.19 M
Total params of ECA_MobileNetV2: 68926.0


In [10]:
# plot_results(macs_mbnet, res_type="macs")

In [11]:
# plot_results(params_mbnet, res_type="params")

### Summary

In [12]:
print(summary(eca_model, input_size=(1, 3, 224, 224)))

Layer (type:depth-idx)                             Output Shape              Param #
ECA_MobileNetV2                                    [1, 2]                    --
├─Sequential: 1-1                                  [1, 128, 14, 14]          --
│    └─ConvBNReLU: 2-1                             [1, 32, 112, 112]         --
│    │    └─Conv2d: 3-1                            [1, 32, 112, 112]         864
│    │    └─BatchNorm2d: 3-2                       [1, 32, 112, 112]         64
│    │    └─ReLU6: 3-3                             [1, 32, 112, 112]         --
│    └─InvertedResidual: 2-2                       [1, 8, 112, 112]          --
│    │    └─Sequential: 3-4                        [1, 8, 112, 112]          627
│    └─InvertedResidual: 2-3                       [1, 16, 56, 56]           --
│    │    └─Sequential: 3-5                        [1, 16, 56, 56]           627
│    └─InvertedResidual: 2-4                       [1, 16, 56, 56]           --
│    │    └─Sequential: 3-6     